In [13]:
import collections
import os
import json
import shutil
import paddle.fluid.dygraph as D
from paddle import fluid
from mindspore.common.tensor import Tensor
input_dir='./'
output_dir='./c'

/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/opt/conda/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [12]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 111.2 MB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 999 kB/s eta 0:00:01
     |████████████████████████████████| 25.2 MB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 145 kB/s eta 0:00:01
     |████████████████████████████████| 82 kB 6.2 MB/s eta 0:00:011
     |████████████████████████████████| 50.4 MB 137 kB/s eta 0:00:01
     |████████████████████████████████| 49 kB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 636 kB 94 kB/s eta 0:00:011
     |████████████████████████████████| 719 kB 168 kB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434674 sha256=583911b7f7622a6e198d21b53572494802df374b4ac244f2cf7c726f4ab191da
  Stored in directory: /tmp/pip-ephem-wheel-cache-jdz2p8pn/wheels/a4/11/6d/2a069739a9961b3127c91a6d792fe8af6f53d93cfc1f234f60
  Created wheel for pathlib: filename=pathlib-1.0.1

In [25]:
def build_params_map(attention_num=12):
    """
    build params map from paddle-paddle's ERNIE to transformer's BERT
    :return:
    """
    weight_map = collections.OrderedDict({
        'word_emb.weight': "bert.bert.bert_embedding_lookup.embedding_table",
        'pos_emb.weight': "bert.bert.bert_embedding_postprocessor.full_position_embedding.embedding_table",
        'sent_emb.weight': "bert.bert.bert_embedding_postprocessor.token_type_embedding.embedding_table",
        'ln.weight': 'bert.bert.bert_embedding_postprocessor.layernorm.gamma',
        'ln.bias': 'bert.bert.bert_embedding_postprocessor.layernorm.beta',
    })
    # add attention layers
    for i in range(attention_num):
        weight_map[f'encoder_stack.block.{i}.attn.q.weight'] = f'bert.bert.bert_encoder.layers.{i}.attention.query_layer.weight'
        weight_map[f'encoder_stack.block.{i}.attn.q.bias'] = f'bert.bert.bert_encoder.layers.{i}.attention.query_layer.bias'
        weight_map[f'encoder_stack.block.{i}.attn.k.weight'] = f'bert.bert.bert_encoder.layers.{i}.attention.key_layer.weight'
        weight_map[f'encoder_stack.block.{i}.attn.k.bias'] = f'bert.bert.bert_encoder.layers.{i}.attention.key_layer.bias'
        weight_map[f'encoder_stack.block.{i}.attn.v.weight'] = f'bert.bert.bert_encoder.layers.{i}.attention.value_layer.weight'
        weight_map[f'encoder_stack.block.{i}.attn.v.bias'] = f'bert.bert.bert_encoder.layers.{i}.attention.value_layer.bias'
        weight_map[f'encoder_stack.block.{i}.attn.o.weight'] = f'bert.bert.bert_encoder.layers.{i}.attention.output.dense.weight'
        weight_map[f'encoder_stack.block.{i}.attn.o.bias'] = f'bert.bert.bert_encoder.layers.{i}.attention.output.dense.bias'
        weight_map[f'encoder_stack.block.{i}.ln1.weight'] = f'bert.bert.bert_encoder.layers.{i}.attention.output.layernorm.gamma'
        weight_map[f'encoder_stack.block.{i}.ln1.bias'] = f'bert.bert.bert_encoder.layers.{i}.attention.output.layernorm.beta'
        weight_map[f'encoder_stack.block.{i}.ffn.i.weight'] = f'bert.bert.bert_encoder.layers.{i}.intermediate.weight'
        weight_map[f'encoder_stack.block.{i}.ffn.i.bias'] = f'bert.bert.bert_encoder.layers.{i}.intermediate.bias'
        weight_map[f'encoder_stack.block.{i}.ffn.o.weight'] = f'bert.bert.bert_encoder.layers.{i}.output.dense.weight'
        weight_map[f'encoder_stack.block.{i}.ffn.o.bias'] = f'bert.bert.bert_encoder.layers.{i}.output.dense.bias'
        weight_map[f'encoder_stack.block.{i}.ln2.weight'] = f'bert.bert.bert_encoder.layers.{i}.output.layernorm.gamma'
        weight_map[f'encoder_stack.block.{i}.ln2.bias'] = f'bert.bert.bert_encoder.layers.{i}.output.layernorm.beta'
    # add pooler
    weight_map.update(
        {
            'pooler.weight': 'bert.pooler.dense.weight',
            'pooler.bias': 'bert.pooler.dense.bias',
            'mlm.weight': 'cls.predictions.transform.dense.weight',
            'mlm.bias': 'cls.predictions.transform.dense.bias',
            'mlm_ln.weight': 'cls.predictions.transform.LayerNorm.gamma',
            'mlm_ln.bias': 'cls.predictions.transform.LayerNorm.beta',
            'mlm_bias': 'cls.predictions.bias'
        }
    )
    return weight_map

In [26]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
print('=' * 20 + 'save config file' + '=' * 20)
config = json.load(open(os.path.join(input_dir, 'ernie_config.json'), 'rt', encoding='utf-8'))
config['layer_norm_eps'] = 1e-5
if 'sent_type_vocab_size' in config:
    config['type_vocab_size'] = config['sent_type_vocab_size']
config['intermediate_size'] = 4 * config['hidden_size']
json.dump(config, open(os.path.join(output_dir, 'config.json'), 'wt', encoding='utf-8'), indent=4)
print('=' * 20 + 'save vocab file' + '=' * 20)
shutil.copyfile(os.path.join(input_dir, 'vocab.txt'), os.path.join(output_dir, 'vocab.txt'))
print('=' * 20 + 'extract weights' + '=' * 20)

====================save config file====================
====================save vocab file====================
====================extract weights====================


In [27]:
state_dict = []
weight_map = build_params_map(attention_num=config['num_hidden_layers'])
with fluid.dygraph.guard():
    paddle_paddle_params, _ = D.load_dygraph(os.path.join(input_dir, 'saved_weights'))
for weight_name, weight_value in paddle_paddle_params.items():
    if 'weight' in weight_name:
        if 'encoder_stack' in weight_name or 'pooler' in weight_name or 'mlm.' in weight_name:
            weight_value = weight_value.transpose()
    state_dict.append({'name':weight_map[weight_name],'data':Tensor(weight_value)})
    print(weight_name, '->', weight_map[weight_name], weight_value.shape)

mlm_bias -> cls.predictions.bias (18000,)
ln.weight -> bert.bert.bert_embedding_postprocessor.layernorm.gamma (768,)
ln.bias -> bert.bert.bert_embedding_postprocessor.layernorm.beta (768,)
word_emb.weight -> bert.bert.bert_embedding_lookup.embedding_table (18000, 768)
pos_emb.weight -> bert.bert.bert_embedding_postprocessor.full_position_embedding.embedding_table (513, 768)
sent_emb.weight -> bert.bert.bert_embedding_postprocessor.token_type_embedding.embedding_table (2, 768)
encoder_stack.block.0.attn.q.weight -> bert.bert.bert_encoder.layers.0.attention.query_layer.weight (768, 768)
encoder_stack.block.0.attn.q.bias -> bert.bert.bert_encoder.layers.0.attention.query_layer.bias (768,)
encoder_stack.block.0.attn.k.weight -> bert.bert.bert_encoder.layers.0.attention.key_layer.weight (768, 768)
encoder_stack.block.0.attn.k.bias -> bert.bert.bert_encoder.layers.0.attention.key_layer.bias (768,)
encoder_stack.block.0.attn.v.weight -> bert.bert.bert_encoder.layers.0.attention.value_layer.we

In [28]:
from mindspore.train.serialization import save_checkpoint

In [29]:
save_checkpoint(state_dict,'2.txt')